In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [21]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

In [2]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 90.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━

In [4]:
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

2025-11-08 12:03:32.471356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762603412.918844      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762603413.053076      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PROJECT_NAME = "Sanskrit_Fine_Tune_Llama_3.1_8B_Instruct"
HF_USER = "Codewgps"


MAX_SEQUENCE_LENGTH = 256

# Run name for saving the model in the hub

RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Hyperparameters for QLoRA

LORA_R = 8
LORA_ALPHA = 8
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# Hyperparameters for Training

EPOCHS = 3 # you can do more epochs if you wish, but only 1 is needed - more is probably overkill
BATCH_SIZE = 1# on an A100 box this can go up to 16
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 2e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# Admin config - note that SAVE_STEPS is how often it will upload to the hub
# I've changed this from 5000 to 2000 so that you get more frequent saves

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline

In [6]:
HUB_MODEL_NAME

'Codewgps/Sanskrit_Fine_Tune_Llama_3.1_8B_Instruct-2025-11-08_12.03.51'

# More on Optimizers

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice

The most common is Adam or AdamW (Adam with Weight Decay).  
Adam achieves good convergence by storing the rolling average of the previous gradients; however, it adds an additional memory footprint of the order of the number of model parameters.


### Log in to HuggingFace and Weights & Biases

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

Repeat this for weightsandbiases at https://wandb.ai and add a secret called `WANDB_API_KEY`

In [8]:
# Log in to HuggingFace
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(hf_token, add_to_git_credential=False)

In [10]:
# Log in to Weights & Biases
wandb_api_key = user_secrets.get_secret("wandb")
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Currently logged in as: poorvik-shrinil (poorvik-shrinil-vit-ap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [11]:
# Load the JSONL dataset
dataset = load_dataset('json', data_files='/kaggle/input/sanskrit-morals-data/sanskrit_moral_stories_finetuning.jsonl', split='train')

print(f"Dataset loaded successfully!")
print(f"Total examples: {len(dataset)}")
print(f"\n Sample entry:")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded successfully!
Total examples: 750

 Sample entry:
{'messages': [{'role': 'user', 'content': 'You are a creative storyteller who specializes in crafting very short, impactful moral stories for all ages based on user input, which will be given in Sanskrit.\n\nIf the input is in Sanskrit, you must first translate it into english and interpret it to understand the intended topic, and then write a short fiction story or folktale (<150 words) with a clear moral.\n\nEach story should have a clear beginning, middle, and end, and end with a 1-line moral or lesson explicitly stated.\n\nSanskrit Input: सत्यं वद, धर्मं चर\nTranslation: Speak the truth, follow righteousness\n\nNow generate a brief, engaging, and unique story with a clear moral.'}, {'role': 'assistant', 'content': "A young potter in a bustling village created beautiful clay vessels. One day, a wealthy merchant came seeking the finest craftsman. Though many offered their wares, the merchant chose the honest potter who 

In [12]:
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Now load the Tokenizer and Model

The model is "quantized" - we are reducing the precision to 4 bits.

In [13]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [14]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Memory footprint: 5591.5 MB


In [ ]:
messages = [
    {"role": "user", "content": """न हि परस्य दुःखं तं शोचति साधुसंमतः।
यः स्वार्थं पश्यति सदा तं जनो न प्रशंसति॥
त्यागेनैव सुखं प्राप्यं न तु लोभेन केनचित्।
यस्य चित्तं शमं याति स एव परमं गतः॥

Now generate a brief, engaging, and unique story with a clear moral."""},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [15]:
# Apply Llama 3.1 Instruct chat template
def formatting_prompts_func(examples):
    """
    Format the dataset using Llama 3.1 Instruct chat template
    """
    texts = []
    for messages in examples["messages"]:
        # Apply chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

print("Dataset formatted with chat template")
print(f"\n Formatted example (first 500 chars):")
print(dataset[0]['text'])

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Dataset formatted with chat template

 Formatted example (first 500 chars):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a creative storyteller who specializes in crafting very short, impactful moral stories for all ages based on user input, which will be given in Sanskrit.

If the input is in Sanskrit, you must first translate it into english and interpret it to understand the intended topic, and then write a short fiction story or folktale (<150 words) with a clear moral.

Each story should have a clear beginning, middle, and end, and end with a 1-line moral or lesson explicitly stated.

Sanskrit Input: सत्यं वद, धर्मं चर
Translation: Speak the truth, follow righteousness

Now generate a brief, engaging, and unique story with a clear moral.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A young potter in a bustling village created beautif

# Data Collator

It's important that we ensure during Training that we are not trying to train the model to predict the description of products; only their price.

We need to tell the trainer that everything up to "Price is $" is there to give context to the model to predict the next token, but does not need to be learned.

The trainer needs to teach the model to predict the token(s) after "Price is $".

There is a complicated way to do this by setting Masks, but luckily HuggingFace provides a super simple helper class to take care of this for us.

In [16]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "<|start_header_id|>assistant<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


# AND NOW

## We set up the configuration for Training

We need to create 2 objects:

A LoraConfig object with our hyperparameters for LoRA

An SFTConfig with our overall Training parameters

In [17]:
# First, specify the configuration parameters for LoRA

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=False,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=False
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters
# The latest version of trl is showing a warning about labels - please ignore this warning
# But let me know if you don't see good training results (loss coming down).

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [18]:
# Fine-tune!
fine_tuning.train()

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Step,Training Loss
50,2.030600
100,0.658300
150,0.128200
200,0.076800
250,0.063700
300,0.056900
350,0.057800
400,0.047200
450,0.046300
500,0.050900


wandb: Adding directory to artifact (Sanskrit_Fine_Tune_Llama_3.1_8B_Instruct-2025-11-08_12.03.51/checkpoint-2000)... Done. 0.2s
wandb: Adding directory to artifact (Sanskrit_Fine_Tune_Llama_3.1_8B_Instruct-2025-11-08_12.03.51/checkpoint-2250)... Done. 0.3s


TrainOutput(global_step=2250, training_loss=0.09360390186309814, metrics={'train_runtime': 11774.7374, 'train_samples_per_second': 0.191, 'train_steps_per_second': 0.191, 'total_flos': 2.5958952341766144e+16, 'train_loss': 0.09360390186309814, 'epoch': 3.0})

In [30]:
# Import required libraries
from transformers import TextStreamer
import torch

# Get the fine-tuned model from trainer
trained_model = fine_tuning.model

# Set model to evaluation mode
trained_model.eval()

# CRITICAL: Ensure model is in BFloat16 (matching training dtype)
trained_model = trained_model.to(torch.bfloat16)

# Create text streamer for live output
text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Test prompt
test_input = """न हि परस्य दुःखं तं शोचति साधुसंमतः।
यः स्वार्थं पश्यति सदा तं जनो न प्रशंसति॥
त्यागेनैव सुखं प्राप्यं न तु लोभेन केनचित्।
यस्य चित्तं शमं याति स एव परमं गतः॥

Now generate a brief, engaging, and unique story with a clear moral."""

# Format as chat
messages = [
    {
        "role": "user",
        "content": f"""You are a creative storyteller who specializes in crafting very short, impactful moral stories for all ages based on user input, which will be given in Sanskrit.

If the input is in Sanskrit, you must first translate it into english and interpret it to understand the intended topic, and then write a short fiction story or folktale (<150 words) with a clear moral.

Each story should have a clear beginning, middle, and end, and end with a 1-line moral or lesson explicitly stated.

Sanskrit Input: {test_input}"""
    }
]

# Apply chat template
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# Generate with streaming and no gradient computation
print("\nTesting inference...\n")

with torch.no_grad():
    outputs = trained_model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print("Inference complete!")



Testing inference...

A scholar lived in a grand library but never shared knowledge with others. A humble student sought wisdom from him repeatedly, and each time, the scholar turned him away rudely. One day, the scholar fell ill. The student, hearing this, brought medicine and nursed him to health. The scholar, touched by this kindness despite his rudeness, finally opened his library and spent his remaining years teaching the grateful student, who became the greatest scholar and spread wisdom generously, just as the student had been.

Moral: Kindness and rudeness are equally rewarded in the long run. Rudeness may drive away friends temporarily, but kindness creates lasting bonds.
Inference complete!


In [20]:
if LOG_TO_WANDB:
  wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▄█▅▄▁▇▁▄▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▆███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,2.5958952341766144e+16
train/epoch,3
train/global_step,2250
train/grad_norm,0.22705
train/learning_rate,0
train/loss,0.0235


In [29]:
print("hello")

hello


In [42]:
from huggingface_hub import login,logout
logout()

In [43]:
from huggingface_hub import login,logout
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("repo")
# Use your taken from https://huggingface.co/settings/tokens
login(token=secret_value_1)

In [44]:
# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved to the hub: Sanskrit_Fine_Tune_Llama_3.1_8B_Instruct-2025-11-08_12.03.51
